In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

### Inconsistent Enforcement of Street Types

* From looking at some of the maps, it seems that some areas area getting ticketed as both business and residential parking
* The most clear way of looking at this is finding tickets given on streets explicitly exempted for business parking for taxis, explicitly defining them as business streets
* Additionally, looking at areas of significant overlap between residential and business tickets might show patterns

In [4]:
res_tickets = pd.read_csv('taxi_residential_tickets.csv')
res_tickets['Issue Date'] = pd.to_datetime(res_tickets['Issue Date'])
res_tickets = res_tickets.set_index(pd.DatetimeIndex(res_tickets['Issue Date']))
print(res_tickets.shape)
print(res_tickets.dtypes)
res_tickets.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(39096, 16)
Ticket Number                     int64
License Plate Number             object
License Plate State              object
License Plate Type               object
Ticket Make                      object
Issue Date               datetime64[ns]
Violation Location               object
Violation Code                   object
Violation Description            object
Badge                            object
Unit                             object
Ticket Queue                     object
Hearing Dispo                    object
Latitude                        float64
Longitude                       float64
Ward                              int64
dtype: object


,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Latitude,Longitude,Ward
2009-01-05 04:19:00,55388795,25495TX,IL,TXI,MERC,2009-01-05 04:19:00,5721 W EASTWOOD,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",13250,16,Notice,Did Not Challenge,41.964930,-87.770331,38
2009-01-18 22:04:00,55984582,3529TX,IL,TXI,MERZ,2009-01-18 22:04:00,2822 W BRYN MAWR,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",19838,20,Paid,Did Not Challenge,41.983193,-87.699957,40
2009-01-19 07:30:00,55772826,1262TX,IL,TXI,FORD,2009-01-19 07:30:00,6320 RIDGE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Paid,Did Not Challenge,42.010156,-87.682322,49
2009-02-08 14:53:00,56126178,1515TX,IL,TXI,FORD,2009-02-08 14:53:00,4219 N MILWAUKEE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",11128,16,Paid,Liable,41.957540,-87.751636,45
2009-02-12 06:10:00,56062981,5410TX,IL,TXI,FORD,2009-02-12 06:10:00,6935 THOME,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Dismissed,DOR Withdraw,41.994834,-87.739869,39


In [7]:
res_tickets['Violation Street'] = res_tickets['Violation Location'].apply(lambda x: re.sub(r'^\d{1,6}\s','',x))
res_tickets['Violation Address Number'] = res_tickets['Violation Location'].apply(lambda x: re.search(r'^\d{1,6}',x).group())
res_tickets['Violation Address Number'] = res_tickets['Violation Address Number'].astype(int)
res_tickets.head()

,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Latitude,Longitude,Ward,Violation Street,Violation Address Number
2009-01-05 04:19:00,55388795,25495TX,IL,TXI,MERC,2009-01-05 04:19:00,5721 W EASTWOOD,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",13250,16,Notice,Did Not Challenge,41.964930,-87.770331,38,W EASTWOOD,5721
2009-01-18 22:04:00,55984582,3529TX,IL,TXI,MERZ,2009-01-18 22:04:00,2822 W BRYN MAWR,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",19838,20,Paid,Did Not Challenge,41.983193,-87.699957,40,W BRYN MAWR,2822
2009-01-19 07:30:00,55772826,1262TX,IL,TXI,FORD,2009-01-19 07:30:00,6320 RIDGE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Paid,Did Not Challenge,42.010156,-87.682322,49,RIDGE,6320
2009-02-08 14:53:00,56126178,1515TX,IL,TXI,FORD,2009-02-08 14:53:00,4219 N MILWAUKEE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",11128,16,Paid,Liable,41.957540,-87.751636,45,N MILWAUKEE,4219
2009-02-12 06:10:00,56062981,5410TX,IL,TXI,FORD,2009-02-12 06:10:00,6935 THOME,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",17392,24,Dismissed,DOR Withdraw,41.994834,-87.739869,39,THOME,6935


#### Business Street Exemption Addresses

* 6000 > N Lincoln >= 5200
* 6100 > N Western >= 6000
* 6000 > N Western >= 5400 (Potentially only odds for East side)
* 3100 > W Peterson >= 2800 (Potentially only odds for South side)
* 2100 > W Devon >= 1900
* 6800 > N Western >= 6200 

In [12]:
ex_lincoln = res_tickets.loc[(res_tickets['Violation Street'].str.contains('N LINCOLN')) & 
                             (res_tickets['Violation Address Number'] < 6000) & 
                             (res_tickets['Violation Address Number'] >= 5200)]
#Keeping both sides in for streets that might only allow one side
ex_western_1 = res_tickets.loc[(res_tickets['Violation Street'].str.contains('N WESTERN')) & 
                               (res_tickets['Violation Address Number'] < 6100) & 
                               (res_tickets['Violation Address Number'] >= 5400)]
ex_western_2 = res_tickets.loc[(res_tickets['Violation Street'].str.contains('N WESTERN')) & 
                               (res_tickets['Violation Address Number'] < 6800) & 
                               (res_tickets['Violation Address Number'] >= 6200)]
ex_peterson = res_tickets.loc[(res_tickets['Violation Street'].str.contains('N PETERSON')) & 
                              (res_tickets['Violation Address Number'] < 2100) & 
                              (res_tickets['Violation Address Number'] >= 1900)]
ex_devon = res_tickets.loc[(res_tickets['Violation Street'].str.contains('W DEVON')) & 
                           (res_tickets['Violation Address Number'] < 2100) & 
                           (res_tickets['Violation Address Number'] >= 1900)]
bus_ex_res = pd.concat([ex_lincoln, ex_western_1, ex_western_2, ex_peterson, ex_devon])
print(bus_ex_res.shape)
bus_ex_res.head()

(502, 18)


,Ticket Number,License Plate Number,License Plate State,License Plate Type,Ticket Make,Issue Date,Violation Location,Violation Code,Violation Description,Badge,Unit,Ticket Queue,Hearing Dispo,Latitude,Longitude,Ward,Violation Street,Violation Address Number
2012-12-11 07:06:00,9183338521,2581TX,IL,TXI,FORD,2012-12-11 07:06:00,5910 N LINCOLN,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",716,498,Paid,Did Not Challenge,41.988433,-87.704189,40,N LINCOLN,5910
2012-12-11 07:08:00,9183338524,5697TX,IL,TXI,CHEV,2012-12-11 07:08:00,5924 N LINCOLN,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",716,498,Paid,Did Not Challenge,41.988723,-87.704721,40,N LINCOLN,5924
2011-07-17 07:46:00,59065036,2307TX,IL,TXI,FORD,2011-07-17 07:46:00,5550 N LINCOLN,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",8280,20,Paid,Did Not Challenge,41.982411,-87.693873,40,N LINCOLN,5550
2012-12-11 07:06:00,9183338522,3218TX,IL,TXI,FORD,2012-12-11 07:06:00,5910 N LINCOLN,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",716,498,Paid,Did Not Challenge,41.988433,-87.704189,40,N LINCOLN,5910
2013-03-30 08:12:00,62672104,1334TX,IL,TXI,FORD,2013-03-30 08:12:00,5546 N LINCOLN AVE,0964170A,"TRUCK,RV,BUS, OR TAXI RESIDENTIAL STREET",7496,20,Dismissed,Not Liable,41.982328,-87.693769,40,N LINCOLN AVE,5546


### Notes

* Looks like 502 tickets were given for residential parking, event though the areas they are issued to seem to be business streets given that they're exempted as such in the regulations